<a href="https://colab.research.google.com/github/juliaeduarda-rg/Dados-Study/blob/main/Desafio_Python_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd
import seaborn as sns
import numpy as np
import json
import requests

In [ ]:
credencial=service_account.Credentials.from_service_account_file(
    '/content/drive/MyDrive/SIRIUS/Dados/Chaves/gabriel-sirius-e7b998baf018.json'
)

In [ ]:
cliente=bigquery.Client(project=credencial.project_id, credentials=credencial)
df_municipios=cliente.query('''SELECT * FROM `basedosdados.br_bd_diretorios_brasil.municipio`''').to_dataframe()

In [ ]:
df_municipios.columns

Index(['id_municipio', 'id_municipio_6', 'id_municipio_tse', 'id_municipio_rf',
       'id_municipio_bcb', 'nome', 'capital_uf', 'id_comarca',
       'id_regiao_saude', 'nome_regiao_saude', 'id_regiao_imediata',
       'nome_regiao_imediata', 'id_regiao_intermediaria',
       'nome_regiao_intermediaria', 'id_microrregiao', 'nome_microrregiao',
       'id_mesorregiao', 'nome_mesorregiao', 'ddd', 'id_uf', 'sigla_uf',
       'nome_uf', 'nome_regiao'],
      dtype='object')

In [ ]:
df_municipios_dir=df_municipios[['id_municipio','nome','nome_uf']]

In [ ]:
df_municipios_dir.head()

,id_municipio,nome,nome_uf
0,1100023,Ariquemes,Rondônia
1,1100106,Guajará-Mirim,Rondônia
2,1100114,Jaru,Rondônia
3,1100130,Machadinho D'Oeste,Rondônia
4,1100205,Porto Velho,Rondônia


In [ ]:
df_pop=cliente.query('''SELECT * FROM `basedosdados.br_ibge_populacao.municipio` where ano=2021''').to_dataframe()
df_pop.head()

,ano,sigla_uf,id_municipio,populacao
0,2021,RO,1100015,22516
1,2021,RO,1100023,111148
2,2021,RO,1100031,5067
3,2021,RO,1100049,86416
4,2021,RO,1100056,16088


In [ ]:
df_pop_tratado=df_pop[['id_municipio','populacao']]

In [ ]:
df_pop_municipios=df_municipios_dir.merge(df_pop_tratado,how='inner',on='id_municipio')
df_pop_municipios.head()

,id_municipio,nome,nome_uf,populacao
0,1100023,Ariquemes,Rondônia,111148
1,1100106,Guajará-Mirim,Rondônia,46930
2,1100114,Jaru,Rondônia,51469
3,1100130,Machadinho D'Oeste,Rondônia,41724
4,1100205,Porto Velho,Rondônia,548952


In [ ]:
df_pop_municipios['Polulacao_Estado']=df_pop_municipios.groupby('nome_uf')['populacao'].transform('sum')
df_pop_municipios.head()

,id_municipio,nome,nome_uf,populacao,Polulacao_Estado
0,1100023,Ariquemes,Rondônia,111148,1815278
1,1100106,Guajará-Mirim,Rondônia,46930,1815278
2,1100114,Jaru,Rondônia,51469,1815278
3,1100130,Machadinho D'Oeste,Rondônia,41724,1815278
4,1100205,Porto Velho,Rondônia,548952,1815278


In [ ]:
df_pop_municipios['percentagem_pop']=round((df_pop_municipios['populacao']/df_pop_municipios['Polulacao_Estado'])*100,4)
df_pop_municipios.head()

,id_municipio,nome,nome_uf,populacao,Polulacao_Estado,percentagem_pop
0,1100023,Ariquemes,Rondônia,111148,1815278,6.1229
1,1100106,Guajará-Mirim,Rondônia,46930,1815278,2.5853
2,1100114,Jaru,Rondônia,51469,1815278,2.8353
3,1100130,Machadinho D'Oeste,Rondônia,41724,1815278,2.2985
4,1100205,Porto Velho,Rondônia,548952,1815278,30.2407


In [ ]:
df_ideb1=cliente.query('''SELECT * FROM `basedosdados.br_inep_ideb.municipio` where ano= 2019''').to_dataframe()

In [ ]:
df_ideb=df_ideb1[['ano','id_municipio','rede','anos_escolares','ideb','ensino']]

#Fazendo o merge df_ideb com o df_pop_municipios

In [ ]:
df_pop_mun_ideb=df_pop_municipios.merge(df_ideb,how='inner',on='id_municipio')

#Lendo as 5 primeiras linhas desse DataFrame  com o metodo head

In [ ]:
df_pop_mun_ideb=df_pop_mun_ideb[(df_pop_mun_ideb.rede=='publica')&(df_pop_mun_ideb['ensino']=='fundamental')]
df_pop_mun_ideb

,id_municipio,nome,nome_uf,populacao,Polulacao_Estado,percentagem_pop,ano,rede,anos_escolares,ideb,ensino
6,1100023,Ariquemes,Rondônia,111148,1815278,6.1229,2019,publica,finais (6-9),5.1,fundamental
8,1100023,Ariquemes,Rondônia,111148,1815278,6.1229,2019,publica,iniciais (1-5),5.2,fundamental
12,1100106,Guajará-Mirim,Rondônia,46930,1815278,2.5853,2019,publica,finais (6-9),4.5,fundamental
15,1100106,Guajará-Mirim,Rondônia,46930,1815278,2.5853,2019,publica,iniciais (1-5),5.0,fundamental
21,1100114,Jaru,Rondônia,51469,1815278,2.8353,2019,publica,finais (6-9),5.1,fundamental
...,...,...,...,...,...,...,...,...,...,...,...
40448,5221080,Teresina de Goiás,Goiás,3538,7206589,0.0491,2019,publica,iniciais (1-5),5.2,fundamental
40451,5221858,Valparaíso de Goiás,Goiás,175720,7206589,2.4383,2019,publica,iniciais (1-5),5.5,fundamental
40454,5221858,Valparaíso de Goiás,Goiás,175720,7206589,2.4383,2019,publica,finais (6-9),4.6,fundamental
40458,5222203,Vila Boa,Goiás,6451,7206589,0.0895,2019,publica,finais (6-9),4.6,fundamental


#Filtrando os anos escolares iniciais e finais no ano de 2019 da rede publica e ensino fundamental

In [ ]:
df_pop_mun_ideb.loc[(df_pop_mun_ideb['ano']==2019)&(df_pop_mun_ideb['ensino']=='fundamental')&(df_pop_mun_ideb['rede']=='publica'),['ideb']]

,ideb
6,5.1
8,5.2
12,4.5
15,5.0
21,5.1
...,...
40448,5.2
40451,5.5
40454,4.6
40458,4.6


#Lendo o dataframe do pib

In [ ]:
df_pib=cliente.query('''SELECT * FROM `basedosdados.br_inep_ideb.brasil` where ano=2019 ''').to_dataframe()

In [ ]:
df_pib

,ano,rede,ensino,anos_escolares,taxa_aprovacao,indicador_rendimento,nota_saeb_matematica,nota_saeb_lingua_portuguesa,nota_saeb_media_padronizada,ideb,projecao
0,2019,estadual,fundamental,finais (6-9),90.0,0.899956,258.02,256.57,5.243500,4.7,5.1
1,2019,estadual,fundamental,iniciais (1-5),96.3,0.964005,230.88,217.55,6.331307,6.1,5.9
2,2019,estadual,medio,todos (1-4),84.5,0.853421,268.60,272.01,4.534282,3.9,4.6
3,2019,municipal,fundamental,finais (6-9),87.3,0.876713,255.57,254.10,5.161167,4.5,4.9
4,2019,municipal,fundamental,iniciais (1-5),93.9,0.939695,223.89,210.77,6.074789,5.7,5.4
5,2019,privada,fundamental,finais (6-9),97.1,0.971498,302.91,293.64,6.609333,6.4,7.1
6,2019,privada,fundamental,iniciais (1-5),98.8,0.988399,253.47,240.71,7.184233,7.1,7.4
7,2019,privada,medio,todos (1-4),96.4,0.966702,334.72,322.10,6.212644,6.0,6.8
8,2019,publica,fundamental,finais (6-9),88.6,0.887787,257.17,255.67,5.214334,4.6,5.0
9,2019,publica,fundamental,iniciais (1-5),94.3,0.943598,222.41,209.05,6.015031,5.7,5.5


In [ ]:
df_pib1=df_pib[['ideb','projecao']]

##Fazendo mais um merge

In [ ]:
df_pib_pop_mun_ideb=df_pop_mun_ideb.merge(df_pib1,how='inner',on='ideb')

In [ ]:
df_pib_pop_mun_ideb

,id_municipio,nome,nome_uf,populacao,Polulacao_Estado,percentagem_pop,ano,rede,anos_escolares,ideb,ensino,projecao
0,1100106,Guajará-Mirim,Rondônia,46930,1815278,2.5853,2019,publica,finais (6-9),4.5,fundamental,4.9
1,1100130,Machadinho D'Oeste,Rondônia,41724,1815278,2.2985,2019,publica,finais (6-9),4.5,fundamental,4.9
2,1100338,Nova Mamoré,Rondônia,32184,1815278,1.7730,2019,publica,finais (6-9),4.5,fundamental,4.9
3,1200138,Bujari,Acre,10572,906876,1.1658,2019,publica,iniciais (1-5),4.5,fundamental,4.9
4,1200336,Mâncio Lima,Acre,19643,906876,2.1660,2019,publica,finais (6-9),4.5,fundamental,4.9
...,...,...,...,...,...,...,...,...,...,...,...,...
3872,4307864,Fagundes Varela,Rio Grande do Sul,2750,11466630,0.0240,2019,publica,iniciais (1-5),7.1,fundamental,7.4
3873,5206800,Damolândia,Goiás,2944,7206589,0.0409,2019,publica,iniciais (1-5),7.1,fundamental,7.4
3874,5215504,Ouvidor,Goiás,6895,7206589,0.0957,2019,publica,iniciais (1-5),7.1,fundamental,7.4
3875,5211602,Ivolândia,Goiás,2295,7206589,0.0318,2019,publica,iniciais (1-5),7.1,fundamental,7.4


#Lendo a base de dados da onu

In [ ]:
df_idhm=cliente.query('''SELECT * FROM `basedosdados.mundo_onu_adh.municipio` where ano=2010''').to_dataframe()

In [ ]:
df_idhm=df_idhm[['id_municipio','expectativa_vida']]

#Fazendo mais um merge

In [ ]:
df_merge_total=df_pib_pop_mun_ideb.merge(df_idhm,how='inner',on='id_municipio')

In [ ]:
df_merge_total

,id_municipio,nome,nome_uf,populacao,Polulacao_Estado,percentagem_pop,ano,rede,anos_escolares,ideb,ensino,projecao,expectativa_vida
0,1100106,Guajará-Mirim,Rondônia,46930,1815278,2.5853,2019,publica,finais (6-9),4.5,fundamental,4.9,74.39
1,1100130,Machadinho D'Oeste,Rondônia,41724,1815278,2.2985,2019,publica,finais (6-9),4.5,fundamental,4.9,70.32
2,1100338,Nova Mamoré,Rondônia,32184,1815278,1.7730,2019,publica,finais (6-9),4.5,fundamental,4.9,71.15
3,1100338,Nova Mamoré,Rondônia,32184,1815278,1.7730,2019,publica,iniciais (1-5),4.9,fundamental,5.2,71.15
4,1200138,Bujari,Acre,10572,906876,1.1658,2019,publica,iniciais (1-5),4.5,fundamental,4.9,71.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3869,4317756,Santo Antônio do Planalto,Rio Grande do Sul,2018,11466630,0.0176,2019,publica,iniciais (1-5),7.1,fundamental,7.4,78.11
3870,4307864,Fagundes Varela,Rio Grande do Sul,2750,11466630,0.0240,2019,publica,iniciais (1-5),7.1,fundamental,7.4,76.14
3871,5206800,Damolândia,Goiás,2944,7206589,0.0409,2019,publica,iniciais (1-5),7.1,fundamental,7.4,74.14
3872,5215504,Ouvidor,Goiás,6895,7206589,0.0957,2019,publica,iniciais (1-5),7.1,fundamental,7.4,74.81


Enviando em forma de arquivo parquet e csv

In [ ]:
df_parquet=df_merge_total.to_parquet('/content/df_merge.parquet')

In [ ]:
df_csv=df_merge_total.to_csv('/content/df_merge.csv')